<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# **SQL Server 2019 big data cluster Tutorial**
## **02 - How to view the status of a big data cluster**

In this tutorial, you'll learn how to access the service endpoints and view the status of a SQL Server big data cluster components. You can use both Azure Data Studio and azdata, and this article covers both techniques. 



## **Use Azure Data Studio**
After downloading the latest insiders build of [Azure Data Studio](https://aka.ms/getazuredatastudio), you can view service endpoints and the status of a big data cluster with the SQL Server big data cluster dashboard. Some of the features below are only first available in the insiders build of Azure Data Studio.

First, create a connection to your big data cluster in Azure Data Studio. For more information, see [Connect to a SQL Server big data cluster with Azure Data Studio](https://docs.microsoft.com/en-us/sql/big-data-cluster/connect-to-big-data-cluster?view=sqlallproducts-allversions).

Right-click on the big data cluster endpoint, and click **Manage**.
![right-click-manage](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/right-click-manage.png)

Select the SQL Server Big Data Cluster tab to access the big data cluster dashboard.
![bdc-dashboard](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/bdc-dashboard.png?view=sqlallproducts-allversions)

### **Service endpoints**
It is important to be able to easily access the various services within a big data cluster. The big data cluster dashboard provides a service endpoints table that allows you to see and copy the service endpoints.

![service-endpoints](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/service-endpoints.png?view=sqlallproducts-allversions)

These services list the endpoints that can be copied and pasted when you need the endpoint for connecting to those services. For example, you can click the copy icon to the right of the endpoint and then paste it in a text window requesting that endpoint. The Cluster Management Service endpoint is necessary to run the [cluster status notebook](https://docs.microsoft.com/en-us/sql/big-data-cluster/view-cluster-status?view=sqlallproducts-allversions#notebook).

### **Dashboards**
The service endpoints table also exposes several dashboards for monitoring:

- Metrics (Grafana)
- Logs (Kibana)
- Spark Job Monitoring
- Spark Resource Management

You can directly click on these links. You will be required to authenticate when accessing these dashboards. For the metrics and logs dashboards, provide controller admin credentials that you set at deployment time using environment variables **AZDATA_USERNAME** and **AZDATA_PASSWORD**. Spark dashboards will use gateway (Knox) credentials: either AD identity in a cluster integrated with AD or user **root** and **AZDATA_PASSWORD** if using basic authentication in your cluster.

### **Cluster Status notebook**


1. You can also view cluster status of the big data cluster by launching the Cluster Status notebook. To launch the notebook, click the Cluster Status task.
![cluster-status-launch](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/cluster-status-launch.png?view=sqlallproducts-allversions)

2. Before you begin, you will need the following items:
- Big data cluster name
- Controller username
- Controller password
- Controller endpoints

The default big data cluster name is **mssql-cluster** unless you customized it during your deployment. You can find the controller endpoint from the big data cluster dashboard in the Service Endpoints table. The endpoint is listed as **Cluster Management Service**. If you do not know the credentials, ask the admin who deployed your cluster.

3. Click **Run Cells** on the top toolbar.

4. Follow the prompt for your credentials. Press press ENTER after you type each credential for the big data cluster name, controller username, and controller password.

5. If you connected successfully, the rest of the notebook will show the output of each component of the big data cluster. When you want to rerun a certain code cell, hover over the code cell and click the **Run** icon.

## **Use azdata**
You can also use [azdata](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-install-azdata?view=sqlallproducts-allversions) commands to view both endpoints and the cluster status.

### **Service endpoints**

Log in to the big data cluster with [azdata login](https://docs.microsoft.com/en-us/sql/big-data-cluster/reference-azdata?view=sqlallproducts-allversions). Set the **--controller-endpoint** parameter to the external IP address of the controller endpoint.

In [ ]:
# azdata login --endpoint https://<ip-address-of-controller-svc-external>:30080 --username <user-name>
azdata login --endpoint https://20.43.145.147:30080 --username ethuang

Specify the username and password that you configured for the controller (AZDATA_USERNAME and AZDATA_PASSWORD) during deployment. For AD authentication, the command is:

In [ ]:
azdata login --endpoint https://<control_domain_name>:30080 --auth ad

Run [azdata bdc endpoint list](https://docs.microsoft.com/en-us/sql/big-data-cluster/reference-azdata-bdc-endpoint?view=sqlallproducts-allversions) to get a list with a description of each endpoint and their corresponding IP address and port values.

In [ ]:
azdata bdc endpoint list -o table

### **View cluster status**
You can view the status of the cluster with the [azdata bdc status](https://docs.microsoft.com/en-us/sql/big-data-cluster/reference-azdata-bdc-status?view=sqlallproducts-allversions) show command.

In [ ]:
azdata bdc status show

### **View specific resource status**
You can view the status of a specific resource within the cluster with the azdata bdc status show command. When you use this command you can filter using --resource parameter. Few examples of inputs for --resource parameter are:

- master
- control
- compute-0
- storage-0
- gateway

For example, the following command displays the status of the storage pool:

In [ ]:
azdata bdc status show --all --resource storage-0

To see the status of all components that are running a specific service you must use the corresponding command group azdata bdc \<serviceName> status show. For example:

- azdata bdc sql status show --all
- azdata bdc hdfs status show --all
- azdata bdc spark status show --all

In [ ]:
azdata bdc spark status show --all

In [ ]:
azdata bdc hdfs status show --all

Here is an sample output:
```
Spark: ready                                                                                                                                                                                                        Health Status:  healthy
===========================================================================================================================================================================================================================================
Resources: ready                                                                                                                                                                                                    Health Status:  healthy
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Resourcename    State    Healthstatus    Details

sparkhead       ready    healthy         StatefulSet sparkhead is healthy
storage-0       ready    healthy         StatefulSet storage-0 is healthy


Sparkhead Resources: running                                                                                                                                                                                        Health Status:  healthy
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Instancename    State    Healthstatus    Details

sparkhead-0     running  healthy         Pod sparkhead-0 is healthy
sparkhead-1     running  healthy         Pod sparkhead-1 is healthy


Dashboards
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Name            Url

nodeMetricsUrl  https://13.91.50.9:30777/api/v1/bdc/instances/sparkhead-1/status/nodemetrics/ui
sqlMetricsUrl   https://13.91.50.9:30777/api/v1/bdc/instances/sparkhead-1/status/sqlmetrics/ui
logsUrl         https://13.91.50.9:30777/api/v1/bdc/instances/sparkhead-1/status/logs/ui


Storage-0 Resources: running                                                                                                                                                                                        Health Status:  healthy
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Instancename    State    Healthstatus    Details

storage-0-0     running  healthy         Pod storage-0-0 is healthy
storage-0-1     running  healthy         Pod storage-0-1 is healthy


Dashboards
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Name            Url

nodeMetricsUrl  https://13.91.50.9:30777/api/v1/bdc/instances/storage-0-1/status/nodemetrics/ui
sqlMetricsUrl   https://13.91.50.9:30777/api/v1/bdc/instances/storage-0-1/status/sqlmetrics/ui
logsUrl         https://13.91.50.9:30777/api/v1/bdc/instances/storage-0-1/status/logs/ui
```

The ```logsUrl``` value links to a Kibana dashboard:
![kibana-dashboard](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/kibana-dashboard.png?view=sqlallproducts-allversions)


The ```nodeMetricsUrl``` and ```sqlMetricsUrl``` values link to a Grafana dashboard for monitoring Kubernetes node metrics and big data cluster service metrics:
![grafana-dashboard](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/grafana-dashboard.png?view=sqlallproducts-allversions)
![grafana-cluster-status](https://docs.microsoft.com/en-us/sql/big-data-cluster/media/view-cluster-status/grafana-sql-status.png?view=sqlallproducts-allversions)

### **View controller status**
You can view the controller status with the [azdata bdc control](https://docs.microsoft.com/en-us/sql/big-data-cluster/reference-azdata-bdc-control-status?view=sqlallproducts-allversions) status show command. It provides similar links to the monitoring dashboards related to the controller components of the big data cluster.